
# Media Bias Detection in News Headlines

This Colab notebook builds a **Media Bias Detector** using **SBERT embeddings + Logistic Regression** and also demonstrates a Hugging Face `datasets`-based workflow, EDA, training, evaluation, and Gradio deployment. It includes code to save model artifacts and prepare a GitHub-ready dashboard.

**Files included in this package:**
- `news_headlines_synthetic_500.csv` (dataset)
- `Media_Bias_Detection_Colab.ipynb` (this notebook)
- `app.py` (Gradio app to deploy locally or on a server)
- `requirements.txt` (Python dependencies)
- `README.md` (repo instructions)

Run this notebook in Google Colab (Runtime -> Change runtime type -> GPU optional but not required).


In [ ]:

# Install required libraries
!pip install -q sentence-transformers scikit-learn gradio datasets transformers seaborn matplotlib pandas spacy joblib
# Download spaCy model for NER (optional)
python -m spacy download en_core_web_sm


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import spacy
from datasets import Dataset
import joblib
import gradio as gr
import os


In [ ]:

# Load synthetic dataset (included in repo)
csv_path = "/mnt/data/media_bias_project/news_headlines_synthetic_500.csv"
df = pd.read_csv(csv_path)
df.head()


In [ ]:

# Basic EDA
print("Dataset shape:", df.shape)
print(df['label'].value_counts())

plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df)
plt.title('Class distribution')
plt.show()

# Word length distribution
df['len'] = df['headline'].apply(lambda x: len(x.split()))
plt.figure(figsize=(6,4))
sns.histplot(df['len'], bins=10)
plt.title('Headline word count distribution')
plt.show()


In [ ]:

# Preprocessing: simple cleaning + optional spaCy NER
import re
import spacy
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_headline'] = df['headline'].apply(clean_text)

# Extract named entities (optional)
def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

df['entities'] = df['headline'].apply(lambda x: extract_entities(x))
df.head()


In [ ]:

# Generate SBERT embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # small & fast
embeddings = model.encode(df['clean_headline'].tolist(), show_progress_bar=True, convert_to_numpy=True)
print("Embeddings shape:", embeddings.shape)

# Create a huggingface Dataset (optional, for HF workflows)
hf_ds = Dataset.from_pandas(df[['headline','clean_headline','label']])
hf_ds = hf_ds.train_test_split(test_size=0.2, seed=42)
hf_ds


In [ ]:

# Train/Test split (we already have hf_ds; use sklearn split for embeddings)
X = embeddings
y = df['label'].values

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42, stratify=y_enc)

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train, y_train)

# Save model and label encoder
os.makedirs('artifacts', exist_ok=True)
joblib.dump(clf, 'artifacts/logreg_sbert.joblib')
joblib.dump(le, 'artifacts/label_encoder.joblib')
model.save('artifacts/sbert_model')  # SentenceTransformer save
print('Saved artifacts to artifacts/')


In [ ]:

# Evaluation
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:

# Quick Gradio demo (run in Colab to launch)
def predict_fn(headline):
    import joblib
    from sentence_transformers import SentenceTransformer
    clf = joblib.load('artifacts/logreg_sbert.joblib')
    le = joblib.load('artifacts/label_encoder.joblib')
    sbert = SentenceTransformer('all-MiniLM-L6-v2')
    clean = lambda t: ''.join([c.lower() for c in t if c.isalnum() or c.isspace()]).strip()
    emb = sbert.encode([clean(headline)])
    pred = clf.predict(emb)[0]
    return le.inverse_transform([pred])[0]

import gradio as gr
iface = gr.Interface(fn=predict_fn, inputs=gr.Textbox(lines=2, placeholder='Enter headline...'), outputs='text', title='Media Bias Detection')
# To launch in Colab uncomment the next line
# iface.launch(share=True)
